In [ ]:
!pip install pyspark
#The Java gateway error might happen if Java is not installed prior to this

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 48.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=48f5c06ad378399767d8115e44e364f4676555230401271e900a9eabe8c4c19a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

#Spark Session สร้างขึ้นมาเพื่อให้ใช้คำส่ง .sql(query) ได้
#query เป็น string ในรูป "SELECT colname FROM tablename"
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True) 
# ถ้าไม่มี header=True มันจะตั้งชื่อคอลัมน์ใหม่ เป็น c_0, c_1, c_2, ...
# แล้วเอาชื่อคอลัมน์เดิม เช่น Year, dep_time, etc. ไปเป็นค่าข้อมูล ห้ามมมมมลืม header=True

from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

# ขั้นตอนต่อไปนี้คือการ register ตาราง flights ลงใน Spark Session
# ต้องมีทุกบรรทัด จะใช้ df_flights = pd.read_csv('flights_small.csv') ไม่ได้
# df ที่มาจาก toPandas จะมีชนิดของข้อมูลเป็น Object ทั้งหมด
# ถ้ามาจาก pd.read_csv มีค่าเป็น float64 จะไม่ผ่านขั้นตอน createDataFrame
df_flights = flights.toPandas()
spark_temp = spark.createDataFrame(df_flights)
spark_temp.createOrReplaceTempView('flights')
print(spark.catalog.listTables())

# สรุป จะใช้ Spark DF มีสองวิธี คือ สร้าง SparkSession ขึ้นมาแล้วใช้ .read.csv (header=True)
# หรือจะ register ตาราง ลงใน catalog (เหมือน SparkSession เป็น sql engine)

Exception: Java gateway process exited before sending its port number

In [ ]:
#ตัวอย่างคำสั่ง sql โดยที่ query = "FROM flights SELECT * LIMIT 10"
spark.sql("FROM flights SELECT * LIMIT 10").show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows



In [ ]:
print("Spark DataFrame")
display(spark.sql("SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"
).show(3))
print('======================')
print("Pandas DataFrame")
display(spark.sql("SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"
).toPandas().head(3))

Spark DataFrame
+------+----+---+
|origin|dest|  N|
+------+----+---+
|   SEA| RNO|  8|
|   SEA| DTW| 98|
|   SEA| CLE|  2|
+------+----+---+
only showing top 3 rows



None

Pandas DataFrame


,origin,dest,N
0,SEA,RNO,8
1,SEA,DTW,98
2,SEA,CLE,2


In [ ]:
#นี้เป็นอีกวิธีในการสร้าง Spark DataFrame แต่จะทำได้ ต้อง register flights เข้า SparkSeesion ก่อน
#ซึ่งทาง DataCamp ใส่มาไว้ก่อนแล้ว

flights = spark.table("flights")
print(flights.show(3))
print("=============================================")

#การสร้างคอลัมน์ใหม่ขึ้นมา ใช้ .withColumn method โดยที่ argument แรกคือชื่อคอลัมน์
#argument ที่สองคือ เอาคอลัมน์ air_time จากตาราง flights มาหารด้วย 60
flights = flights.withColumn("duration_hrs", flights.air_time/60)
# flights = flights.withColumn("duration_hrs", flights['air_time']/60) ก็ได้
#flights['air_time'] ไม่สามารถใช้แทนได้
print(flights.show(3))

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows

None
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------

In [ ]:
display(spark.sql("SELECT origin, dest, air_time / 60 FROM flights").show(3))

#Doing the same thing in Pandas
df_flights['duration_hrs'] = df_flights['air_time']/60
display(df_flights[["origin", "dest", 'duration_hrs']].head(3))


+------+----+-------------------------------+
|origin|dest|(air_time / CAST(60 AS DOUBLE))|
+------+----+-------------------------------+
|   SEA| LAX|                            2.2|
|   SEA| HNL|                            6.0|
|   SEA| SFO|                           1.85|
+------+----+-------------------------------+
only showing top 3 rows



None

,origin,dest,duration_hrs
0,SEA,LAX,2.20
1,SEA,HNL,6.00
2,SEA,SFO,1.85


In [ ]:
display(spark.sql("SELECT COUNT(*) FROM flights GROUP BY origin").show())

#Doing the same thing in Pandas
display(df_flights.groupby('origin').agg({'year':'count'}))

+--------+
|count(1)|
+--------+
|    6754|
|    3246|
+--------+



None

,year
origin,
PDX,3246
SEA,6754


In [ ]:
display(spark.sql('SELECT origin, dest, COUNT(*) FROM flights GROUP BY origin, dest').show(5))

#Doing the same thing in Pandas
display(df_flights.groupby(['origin', 'dest']).agg({'year':'count'}).loc[[('SEA', 'RNO'), ('SEA','DTW'), ('SEA','CLE'),('SEA', 'LAX'), ('PDX', 'SEA')]])

+------+----+--------+
|origin|dest|count(1)|
+------+----+--------+
|   SEA| RNO|       8|
|   SEA| DTW|      98|
|   SEA| CLE|       2|
|   SEA| LAX|     450|
|   PDX| SEA|     144|
+------+----+--------+
only showing top 5 rows



None

year
origin dest      
SEA    RNO      8
       DTW     98
       CLE      2
       LAX    450
PDX    SEA    144

In [ ]:
display(spark.sql("SELECT AVG(air_time) / 60 FROM flights GROUP BY origin, carrier").show(5))

#doing the same thing in Pandas, but do not know why it is not similar
display((df_flights.groupby(['origin', 'carrier'])[["air_time"]].mean()/60).head())

+------------------------------------+
|(avg(air_time) / CAST(60 AS DOUBLE))|
+------------------------------------+
|                   2.641062801932367|
|                                 NaN|
|                                 NaN|
|                                 NaN|
|                                 NaN|
+------------------------------------+
only showing top 5 rows



None

air_time
origin carrier          
PDX    AA           3.27
       AS           2.56
       B6           3.01
       DL           2.96
       F9           2.13

In [ ]:
long_flights1 = flights.filter("distance > 1000")
long_flights2 = flights.filter(flights.distance > 1000)
display(long_flights1.show(3))
display(long_flights2.show(3))

#Doing the same thing in Pandas
display(df_flights[df_flights["distance"] > 1000].head(3))

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|  10|    40|         6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|   135.0|    1050|  12|    36|        2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|   198.0|    1721|  18|    12|         3.3|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



None

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|  10|    40|         6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|   135.0|    1050|  12|    36|        2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|   198.0|    1721|  18|    12|         3.3|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



None

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.00,2677,10,40,6.00
8,2014,4,19,1236,-4,1508,-7,AS,N309AS,490,SEA,SAN,135.00,1050,12,36,2.25
9,2014,11,19,1812,-3,2352,-4,AS,N564AS,26,SEA,ORD,198.00,1721,18,12,3.30


In [ ]:
# Select the first set of columns
selected1 = flights.select("tailnum", "origin", "dest")

# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)
display(selected1.show(3))
display(selected2.show(3))

#Doing the same thing in Pandas
print("================================")
display(df_flights[["tailnum", "origin", "dest"]].head(3))
display(df_flights[(df_flights["origin"]=="SEA") & (df_flights["dest"]=="PDX")][["origin", "dest", "carrier"]].head(3))

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
+-------+------+----+
only showing top 3 rows



None

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 3 rows



None

,tailnum,origin,dest
0,N846VA,SEA,LAX
1,N559AS,SEA,HNL
2,N847VA,SEA,SFO


,origin,dest,carrier
140,SEA,PDX,OO
207,SEA,PDX,OO
211,SEA,PDX,OO


In [ ]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")
display(speed1.show(3))
display(speed2.show(3))

#Doing the same thing in Pandas
df_flights["avg_speed"] = df_flights["distance"]/(df_flights["air_time"]/60)
display(df_flights[["origin", "dest", "tailnum", "avg_speed"]].head(3))

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



None

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



None

,origin,dest,tailnum,avg_speed
0,SEA,LAX,N846VA,433.64
1,SEA,HNL,N559AS,446.17
2,SEA,SFO,N847VA,367.03


In [ ]:
#The first 3 lines must be typed to convert the distance and air_time columns to interger
from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

# Find the shortest flight from PDX in terms of distance
flights.filter(flights.origin=="PDX").groupBy().min("distance").show()

# Find the longest flight from SEA in terms of air time
flights.filter(flights.origin=="SEA").groupBy().max("air_time").show()

#Doing the same thing with Pandas DataFrame
print(df_flights.groupby('origin')["distance"].get_group("PDX").min())
print(df_flights.groupby('origin')["air_time"].get_group("SEA").max())

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+

106
409.0


In [ ]:
#ต้องใส่มา เพราะระหว่างทางเกิดการแก้ไข flights table?
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)

from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

# Average duration of Delta flights
# ใน Spark DataFrame จะเรียก aggregation methods เช่น max ต้องเรียก .groupBy().max("ชื่อคอลัมน์")
# .groupBy() ในที่นี้ ยังไม่ต้องมี argument ข้างใน ไม่เหมือน pd.DataFrame.groupby() ที่ต้องมีชื่อ
# .groupBy() ในที่นี้ มีไว้เพียงเพราะเป็น Syntax ในการเรียก aggregation เฉยๆ แต่เดี๋ยวจะมีแบบใส่ชื่อด้วย
flights.filter(flights.origin=="SEA").filter(flights.carrier=='DL').groupBy().avg("air_time").show()

# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show()

#Doing the same thing with Pandas DataFrame
display(df_flights.groupby(["origin", "carrier"]).mean().loc[("SEA", "DL"), "air_time"])

# ตัวอย่างนี้แสดงให้เห็นว่า .groupBy ของ Spark df ไม่ได้ใช้ group อะไรเลยด้วยซ้ำ
display(df_flights["duration_hrs"].sum())

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



188.20689655172413

25289.6

In [ ]:
#ต้องใส่มา เพราะระหว่างทางเกิดการแก้ไข flights table?
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)

from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

# # Number of flights each plane made grouped by tailnum 
display(flights.groupBy("tailnum").count().show(3))

# Average duration of flights from PDX and SEA grouped by origin
display(flights.groupBy("origin").avg("air_time").show())

#Doing the same thing in Pandas
display(df_flights.groupby("tailnum").agg({"tailnum":"count"}).loc[["N442AS", "N102UW", "N36472"]].head(3))
display(df_flights.groupby("origin")[["air_time"]].mean())

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
+-------+-----+
only showing top 3 rows



None

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



None

,tailnum
tailnum,
N442AS,38
N102UW,2
N36472,4


,air_time
origin,
PDX,137.12
SEA,160.44


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)
from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# Average departure delay by month and destination grouped by month and dest
display(flights.groupBy("month", "dest").avg("dep_delay").show(3))

# Standard deviation of departure delay
display(flights.groupBy("month", "dest").agg(F.stddev("dep_delay")).show(3))

# Doing the same thing in Pandas
display(df_flights.groupby(["month", "dest"]).agg({"dep_delay":"mean"}).loc[[("11", "TUS"),("11", "ANC"), ("1", "BUR")]])
display(df_flights.groupby(["month", "dest"]).agg({"dep_delay":"std"}).loc[[("11", "TUS"),("11", "ANC"), ("1", "BUR")]])

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|   11| TUS|-2.3333333333333335|
|   11| ANC|  7.529411764705882|
|    1| BUR|              -1.45|
+-----+----+-------------------+
only showing top 3 rows



None

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|   11| TUS|    3.0550504633038935|
|   11| ANC|    18.604716401245316|
|    1| BUR|     15.22627576540667|
+-----+----+----------------------+
only showing top 3 rows



None

dep_delay
month dest           
11    TUS       -2.33
      ANC        7.53
1     BUR       -1.45

dep_delay
month dest           
11    TUS        3.06
      ANC       18.60
1     BUR       15.23

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)
airports = spark.read.csv('airports.csv', header=True)
from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

# Examine the data
print(airports.show(3))

# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")

# Join the DataFrames
flights_with_airports = flights.join(airports, on="dest", how='leftouter')

# Examine the new DataFrame
print(flights_with_airports.show())

# Doing similar things in Pandas
df_airports = pd.read_csv("airports.csv")
df_flights = pd.read_csv("flights_small.csv")
df_airports = df_airports.rename(columns={"faa":"dest"})
df_flights = df_flights.set_index("dest")
df_airports = df_airports.set_index("dest")

# .join() method ของ Pandas คนละอันกับ .join() ของ PySpark
display(df_flights.join(df_airports, on='dest', how='left').head(20))

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows

None
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+--------------------+---------+-----------+----+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|                name|      lat|        lon| alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+--------------------+---------+-----------+----+---+---+
|

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,air_time,distance,hour,minute,name,lat,lon,alt,tz,dst
dest,,,,,,,,,,,,,,,,,,,,,
LAX,2014,12,8,658.00,-7.00,935.00,-5.00,VX,N846VA,1780,SEA,132.00,954,6.00,58.00,Los Angeles Intl,33.94,-118.41,126,-8,A
HNL,2014,1,22,"1,040.00",5.00,"1,505.00",5.00,AS,N559AS,851,SEA,360.00,2677,10.00,40.00,Honolulu Intl,21.32,-157.92,13,-10,N
SFO,2014,3,9,"1,443.00",-2.00,"1,652.00",2.00,VX,N847VA,755,SEA,111.00,679,14.00,43.00,San Francisco Intl,37.62,-122.37,13,-8,A
SJC,2014,4,9,"1,705.00",45.00,"1,839.00",34.00,WN,N360SW,344,PDX,83.00,569,17.00,5.00,Norman Y Mineta San Jose Intl,37.36,-121.93,62,-8,A
BUR,2014,3,9,754.00,-1.00,"1,015.00",1.00,AS,N612AS,522,SEA,127.00,937,7.00,54.00,Bob Hope,34.20,-118.36,778,-8,A
DEN,2014,1,15,"1,037.00",7.00,"1,352.00",2.00,WN,N646SW,48,PDX,121.00,991,10.00,37.00,Denver Intl,39.86,-104.67,5431,-7,A
OAK,2014,7,2,847.00,42.00,"1,041.00",51.00,WN,N422WN,1520,PDX,90.00,543,8.00,47.00,Metropolitan Oakland Intl,37.72,-122.22,9,-8,A
SFO,2014,5,12,"1,655.00",-5.00,"1,842.00",-18.00,VX,N361VA,755,SEA,98.00,679,16.00,55.00,San Francisco Intl,37.62,-122.37,13,-8,A
SAN,2014,4,19,"1,236.00",-4.00,"1,508.00",-7.00,AS,N309AS,490,SEA,135.00,1050,12.00,36.00,San Diego Intl,32.73,-117.19,17,-8,A


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights = spark.read.csv('flights_small.csv', header=True)
airports = spark.read.csv('airports.csv', header=True)
planes = spark.read.csv('planes.csv', header=True)
from pyspark.sql.types import IntegerType
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

# Rename year column
planes = planes.withColumnRenamed("year", "plane_year")

# Join the DataFrames
model_data = flights.join(planes, on="tailnum", how="leftouter")
model_data.show(4)

# Doing the same thing in Pandas
df_airports = pd.read_csv("airports.csv")
df_flights = pd.read_csv("flights_small.csv")
df_planes = pd.read_csv("planes.csv")

df_airports = df_airports.rename(columns={"faa":"dest"})
df_planes = df_planes.rename(columns={"year":"plane_year"})
df_flights = df_flights.set_index("tailnum")
df_planes = df_planes.set_index("tailnum")

# .join() method ของ Pandas คนละอันกับ .join() ของ PySpark
display(df_flights.join(df_planes, on='tailnum', how='left').head(4))
df_model = df_flights.join(df_planes, on='tailnum', how='left')

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   SEA| HNL|     360|    2677|  10|    40|      2006|Fixed wing multi ...|      BOEING| 737-890|      2|  1

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,flight,origin,dest,air_time,distance,hour,minute,plane_year,type,manufacturer,model,engines,seats,speed,engine
tailnum,,,,,,,,,,,,,,,,,,,,,,,
N846VA,2014,12,8,658.00,-7.00,935.00,-5.00,VX,1780,SEA,LAX,132.00,954,6.00,58.00,"2,011.00",Fixed wing multi engine,AIRBUS,A320-214,2.00,182.00,nan,Turbo-fan
N559AS,2014,1,22,"1,040.00",5.00,"1,505.00",5.00,AS,851,SEA,HNL,360.00,2677,10.00,40.00,"2,006.00",Fixed wing multi engine,BOEING,737-890,2.00,149.00,nan,Turbo-fan
N847VA,2014,3,9,"1,443.00",-2.00,"1,652.00",2.00,VX,755,SEA,SFO,111.00,679,14.00,43.00,"2,011.00",Fixed wing multi engine,AIRBUS,A320-214,2.00,182.00,nan,Turbo-fan
N360SW,2014,4,9,"1,705.00",45.00,"1,839.00",34.00,WN,344,PDX,SJC,83.00,569,17.00,5.00,"1,992.00",Fixed wing multi engine,BOEING,737-3H4,2.00,149.00,nan,Turbo-fan


In [ ]:
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast("integer"))
model_data = model_data.withColumn("air_time", model_data.air_time.cast("integer"))
model_data = model_data.withColumn("month", model_data.month.cast("integer"))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast("integer"))



In [ ]:
model_data = model_data.withColumn("plane_age", model_data.year-model_data.plane_year)

# Doing the same thing in Pandas
df_model["plane_age"] = df_model["year"]-df_model["plane_year"]

In [ ]:
# Create is_late
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)

# Convert to an integer
model_data = model_data.withColumn("label", model_data.is_late.cast("integer"))

# Remove missing values
model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL")

# Doing the same thing in Pandas
df_model = df_flights.join(df_planes, on='tailnum', how='left')
df_model["is_late"] = df_model["arr_delay"]>0
df_model["label"] = df_model["is_late"].astype(int)

# df_model.dropna() cannot be used.  It will result only in 5 observations.
df_model = df_model[(~df_model["arr_delay"].isnull()) & (~df_model["dep_delay"].isnull()) & (~df_model["air_time"].isnull()) & (~df_model["plane_year"].isnull())]
display(df_model.head(20))
display(model_data.show())

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,flight,origin,dest,air_time,distance,hour,minute,plane_year,type,manufacturer,model,engines,seats,speed,engine,is_late,label
tailnum,,,,,,,,,,,,,,,,,,,,,,,,,
N846VA,2014,12,8,658.00,-7.00,935.00,-5.00,VX,1780,SEA,LAX,132.00,954,6.00,58.00,"2,011.00",Fixed wing multi engine,AIRBUS,A320-214,2.00,182.00,nan,Turbo-fan,False,0
N559AS,2014,1,22,"1,040.00",5.00,"1,505.00",5.00,AS,851,SEA,HNL,360.00,2677,10.00,40.00,"2,006.00",Fixed wing multi engine,BOEING,737-890,2.00,149.00,nan,Turbo-fan,True,1
N847VA,2014,3,9,"1,443.00",-2.00,"1,652.00",2.00,VX,755,SEA,SFO,111.00,679,14.00,43.00,"2,011.00",Fixed wing multi engine,AIRBUS,A320-214,2.00,182.00,nan,Turbo-fan,True,1
N360SW,2014,4,9,"1,705.00",45.00,"1,839.00",34.00,WN,344,PDX,SJC,83.00,569,17.00,5.00,"1,992.00",Fixed wing multi engine,BOEING,737-3H4,2.00,149.00,nan,Turbo-fan,True,1
N612AS,2014,3,9,754.00,-1.00,"1,015.00",1.00,AS,522,SEA,BUR,127.00,937,7.00,54.00,"1,999.00",Fixed wing multi engine,BOEING,737-790,2.00,151.00,nan,Turbo-jet,True,1
N646SW,2014,1,15,"1,037.00",7.00,"1,352.00",2.00,WN,48,PDX,DEN,121.00,991,10.00,37.00,"1,997.00",Fixed wing multi engine,BOEING,737-3H4,2.00,149.00,nan,Turbo-fan,True,1
N422WN,2014,7,2,847.00,42.00,"1,041.00",51.00,WN,1520,PDX,OAK,90.00,543,8.00,47.00,"2,002.00",Fixed wing multi engine,BOEING,737-7H4,2.00,140.00,nan,Turbo-fan,True,1
N361VA,2014,5,12,"1,655.00",-5.00,"1,842.00",-18.00,VX,755,SEA,SFO,98.00,679,16.00,55.00,"2,013.00",Fixed wing multi engine,AIRBUS,A320-214,2.00,182.00,nan,Turbo-fan,False,0
N309AS,2014,4,19,"1,236.00",-4.00,"1,508.00",-7.00,AS,490,SEA,SAN,135.00,1050,12.00,36.00,"2,001.00",Fixed wing multi engine,BOEING,737-990,2.00,149.00,nan,Turbo-jet,False,0


+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|plane_age|is_late|label|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|      3.0|  false|    0|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     A

None

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Create a StringIndexer
carr_indexer = StringIndexer(inputCol="carrier", outputCol="carrier_index")

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="carrier_index", outputCol="carrier_fact")

# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="dest", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

# Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol="features")

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])
piped_data = flights_pipe.fit(model_data).transform(model_data)
training, test = piped_data.randomSplit([0.6, 0.4])
# This is something beyond Pandas so I don't write it here.

In [ ]:
piped_data.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|plane_age|is_late|label|dest_index|      dest_fact|carrier_index|  carrier_fact|            features|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|

In [ ]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

In [ ]:
# Import the evaluation submodule
import numpy as np 
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)

In [ ]:
# Call lr.fit()
best_lr = lr.fit(training)

# Print best_lr
print(best_lr)

LogisticRegressionModel: uid=LogisticRegression_586ad42037f7, numClasses=2, numFeatures=81


In [ ]:
# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6992038625200666
